In [10]:
import pickle
import numpy as np
import os
from music21 import instrument, note, stream, chord

import parsing_midi
import models

In [19]:
#the file of learned weights we wish to use
weight_file = 'weights-improvement-156-0.2790-bigger.hdf5'
#length of output to generate
output_lenght = 500
#length of input sequence of notes to generate the next note
#make sure this matches the model the weights were trained on!
sequence_length = 100
#notesfile, has a list of all the different notes in the training set
#this needs to match the model the weights were trained on
notefile = 'notes'

In [20]:
#This section runs the model once the weights are generated
#generates an output (numerical output)
def generate_notes(model, network_input, pitchnames, n_vocab):
    #use a random input set as the seed
    start = np.random.randint(0, len(network_input)-1)  
    
    #pattern is the initial melody seed we will input
    pattern = network_input[start]
    
    prediction_output = []
    
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # generate some number of notes
    for note_index in range(output_lenght):
        #aligning input pattern with model input size
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        
        #predict the next note
        prediction = model.predict(prediction_input, verbose=0)
        #end result of neural network is a list of probabilities this is the next note
        #grab note with highest probability
        index = np.argmax(prediction)
        result = int_to_note[index]
        
        #add predicted note to our output
        prediction_output.append(result)
        
        #add predicted note to end of input pattern and cut off begining of pattern
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
    
    return prediction_output

In [21]:
#This section decodes the RNN friendly note format
def decode (prediction_output):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5
        
    return output_notes

In [22]:
#This section creates the midi file
def make_midi(output_notes):
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')

In [23]:
#Main function to cover using the model to generate a midi file
def generate():
    #load the notes used to train the model
    if os.path.isfile('data/'+notefile):
        with open('data/'+notefile, 'rb') as filepath:
            notes = pickle.load(filepath)
            print("load notes from file")
    else:
        notes = parsing_midi.get_notes()
        print("load notes from function")
    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = parsing_midi.prepare_sequence_in(notes, n_vocab, sequence_length)
    
    model = models.create_lstm_network(normalized_input, n_vocab)
    model.load_weights(weight_file)
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    make_midi(decode(prediction_output))

In [13]:
#if called from commandline, this should just try rnning the network
#if called in a notebook, will just run
if __name__ == '__main__':
    generate()

load notes from file
